# Introduction to SQLAlchemy

- __0.__ Introduction
- __1.__ Importing necessary libraries
- __2.__ Creating engine
- __3.__ Creating the connection
- __4.__ Inspecting the database
- __5.__ Querying the database
- __6.__ Read a View to DataFrame
- __7.__ Example of Using a CTE
- __8.__ Create a New Table from DataFrame
- __9.__ Delete a table
- __10.__ Workflow example

## 0. Introduction
SQLAlchemy is a library that facilitates communication between Python programs and databases. 

There are two ways of working with SQLAlchemy:
- SQLAlchemy Core: the foundational architecture for SQLAlchemy as a “database toolkit”. It provides tools for managing connectivity to a database, interacting with database queries and results, and construction of SQL statements
- SQLAlchemy ORM:  builds upon the Core to provide optional object-relational mapping capabilities as it represents database relations as Python objects.

In this course, we will focus on the SQLAlchemy Core architecture, which gives us the following advantages:
- Learning the basic concepts in SQLAlchemy
- practicing the use of the Pandas library
- practicing the use of SQL statements

In [1]:
!pip install sqlalchemy
!pip install pyodbc

## 1. Importing necessary libraries

In [2]:
from sqlalchemy import create_engine, Table, inspect
import pandas as pd

## 2. Creating engine

In order to connect to a database, first we need to create an engine. The engine references are:
- a __Dialect__: There are several types of RDBMS, which we call dialects in SQLAlchemy. They all use SQL as a base, but they have slight differences. \
The most common RDBMS are: MSSQL, MySQL, PostgreSQL, Oracle, SQLite. \
Each dialect in SQLAlchemy has a standard DBAPI, which serves as a bridge between Python programs and the relational databases, and they standardize the way to perform database operations.
- a __Pool__: a pool will establish a connection at the specified server location
- a __Database__: the database name we want to connect to
- __username__ and __password__: when using SQL Server identication, we need to provide the username and password to be able to connect to the database. When using the Microsoft Authentication method, the username and password is not required, as the user is automatically identified by the Microsoft signin credentials. 

__NOTE__: by creating an engine, we did not yet connect to the database, we simply gave the instructions of how and where to connect.

In [3]:
def new_engine(dialect, server, database, user=None, password=None, integrated_security=True):
    if integrated_security:
        # For Windows authentication
        eng = f"{dialect}://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
    else:
        # For SQL Server authentication
        eng = f"{dialect}://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    print(eng)
    return create_engine(eng)

In [4]:
# For Windows authentication
# Replace the server argument with the Server Name found when logging into SQL Server Management Studio 
# OR
# For SQL Server authentication
# Replace the server, user and password argument with the Server Name, username and password
engine = new_engine('mssql', 'DESKTOP-CIKOHBH', 'AdventureWorks2022', integrated_security=True)

mssql://DESKTOP-CIKOHBH/AdventureWorks2022?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server


In [5]:
print(type(engine))

<class 'sqlalchemy.engine.base.Engine'>


## 3. Creating the connection

The engine class instance we created has the .connect() method, which returns a Connection object and technically creates the connection between the database and our Python application.

In [6]:
connection = engine.connect()

In [7]:
print(type(connection))

<class 'sqlalchemy.engine.base.Connection'>


## 4. Inspecting the database

Now that we have a connection to the database, we can inspect its contents. \
The AdventureWorks database has multiple schemas which we can list with the following command.

In [8]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()
print(schemas)

['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'HumanResources', 'INFORMATION_SCHEMA', 'Person', 'Production', 'Purchasing', 'Sales', 'sys']


In [9]:
for schema in schemas:
    print(schema)

db_accessadmin
db_backupoperator
db_datareader
db_datawriter
db_ddladmin
db_denydatareader
db_denydatawriter
db_owner
db_securityadmin
dbo
guest
HumanResources
INFORMATION_SCHEMA
Person
Production
Purchasing
Sales
sys


Each schema in the database logically groups together tables, views, indexes and procedures. Schemas also help with security, as we can grant permission for users into specific schemas. \
In this example we list all the available tables in the Sales schema. All these tables are related to sales or the sales department.

In [10]:
print(inspector.get_table_names(schema='Sales'))

['August_Goal_Table', 'CountryRegionCurrency', 'CreditCard', 'Currency', 'CurrencyRate', 'Customer', 'PersonCreditCard', 'SalesOrderDetail', 'SalesOrderHeader', 'SalesOrderHeaderSalesReason', 'SalesPerson', 'SalesPersonQuotaHistory', 'SalesReason', 'SalesTaxRate', 'SalesTerritory', 'SalesTerritoryHistory', 'ShoppingCartItem', 'SpecialOffer', 'SpecialOfferProduct', 'Store']


In [13]:
for col in inspector.get_columns(table_name='Customer', schema='Sales'):
#     print(col)
    print(col['name'])

CustomerID
PersonID
StoreID
TerritoryID
AccountNumber
rowguid
ModifiedDate


## 5. Querying the database

The Pandas library can directly connect to and query a database with the .read_sql() method. The two most important arguments of the method are:
- sql: this is the SQL command that the query will execute  
- con: the connection we defined to the database

The result is a Pandas DataFrame.

In [14]:
df = pd.read_sql(sql="SELECT * from Sales.Customer", con=connection)

In [15]:
df.head(12)

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
5,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263
6,7,NaN,930.0,1,AW00000007,03E9273E-B193-448E-9823-FE0C44AEED78,2014-09-12 11:15:07.263
7,8,NaN,1024.0,5,AW00000008,801368B1-4323-4BFA-8BEA-5B5B1E4BD4A0,2014-09-12 11:15:07.263
8,9,NaN,620.0,5,AW00000009,B900BB7F-23C3-481D-80DA-C49A5BD6F772,2014-09-12 11:15:07.263
9,10,NaN,928.0,6,AW00000010,CDB6698D-2FF1-4FBA-8F22-60AD1D11DABD,2014-09-12 11:15:07.263


The SQL query can get as complex as the user wants.\
Here is an example of multiple JOIN statements

In [16]:
multijoin_sql ="""
SELECT
  c.CustomerID, 
  c.StoreID, 
  c.AccountNumber, 
  soh.SalesOrderID, 
  sod.ProductID
FROM Sales.Customer AS c INNER JOIN Sales.SalesOrderHeader AS soh 
    ON c.CustomerID = soh.CustomerID
INNER JOIN Sales.SalesOrderDetail AS sod 
    ON soh.SalesOrderID = sod.SalesOrderID
"""
join_df = pd.read_sql(sql=multijoin_sql, con=connection)

In [17]:
join_df.head()

,CustomerID,StoreID,AccountNumber,SalesOrderID,ProductID
0,29580,518.0,AW00029580,43665,707
1,29614,592.0,AW00029614,43668,707
2,29844,1086.0,AW00029844,43673,707
3,29824,1044.0,AW00029824,43677,707
4,29889,1184.0,AW00029889,43678,707


## 6. Read a View to DataFrame
Reading in Views into a DataFrame is the exact same process as reading the tables, we just need to specify which view to read

In [18]:
view_sql = """
SELECT * 
FROM HumanResources.vEmployee
"""

view = pd.read_sql(sql=view_sql, con=connection)

In [19]:
view.head()

,BusinessEntityID,Title,FirstName,MiddleName,LastName,Suffix,JobTitle,PhoneNumber,PhoneNumberType,EmailAddress,EmailPromotion,AddressLine1,AddressLine2,City,StateProvinceName,PostalCode,CountryRegionName,AdditionalContactInfo
0,250,None,Sheela,H,Word,None,Purchasing Manager,210-555-0193,Work,sheela0@adventure-works.com,0,535 Greendell Pl,None,Sammamish,Washington,98074,United States,None
1,249,None,Wendy,Beth,Kahn,None,Finance Manager,248-555-0134,Work,wendy0@adventure-works.com,2,4525 Benedict Ct.,None,Sammamish,Washington,98074,United States,None
2,248,None,Mike,K,Seamans,None,Accountant,927-555-0150,Work,mike0@adventure-works.com,2,1245 Clay Road,None,Index,Washington,98256,United States,None
3,247,None,Janet,L,Sheperdigian,None,Accounts Payable Specialist,393-555-0186,Work,janet0@adventure-works.com,0,6871 Thornwood Dr.,None,Sammamish,Washington,98074,United States,None
4,246,None,Dragan,K,Tomic,None,Accounts Payable Specialist,117-555-0185,Work,dragan0@adventure-works.com,1,3884 Beauty Street,# 14,Gold Bar,Washington,98251,United States,None


## 7. Example of Using a CTE (Common Table Expression)
Common Table Expressions are a great tool for managing more complex queries.\
Basically we create intermediate temporary tables, which we use later in further queries.

In [20]:
cte_sql = """
WITH CTE_Employee AS (
  SELECT 
    P.BusinessEntityID, 
    P.FirstName, 
    P.LastName, 
    HR.JobTitle 
  FROM HumanResources.Employee AS HR INNER JOIN Person.Person AS P 
      ON HR.BusinessEntityID = P.BusinessEntityID 
  WHERE 
    Gender = 'M'
) 
SELECT 
  CTE_Employee.BusinessEntityID, 
  CTE_Employee.FirstName, 
  CTE_Employee.LastName, 
  CTE_Employee.JobTitle, 
  SUM(Sales.SubTotal) TotalSum 
FROM CTE_Employee INNER JOIN Sales.SalesOrderHeader AS Sales 
    ON CTE_Employee.BusinessEntityID = Sales.SalesPersonID 
GROUP BY 
  CTE_Employee.BusinessEntityID, 
  CTE_Employee.FirstName, 
  CTE_Employee.LastName, 
  CTE_Employee.JobTitle 
ORDER BY 
  CTE_Employee.BusinessEntityID;

"""

result = pd.read_sql(cte_sql, engine)
result

,BusinessEntityID,FirstName,LastName,JobTitle,TotalSum
0,274,Stephen,Jiang,North American Sales Manager,1.092124e+06
1,275,Michael,Blythe,Sales Representative,9.293903e+06
2,278,Garrett,Vargas,Sales Representative,3.609447e+06
3,279,Tsvi,Reiter,Sales Representative,7.171013e+06
4,281,Shu,Ito,Sales Representative,6.427006e+06
5,282,José,Saraiva,Sales Representative,5.926418e+06
6,283,David,Campbell,Sales Representative,3.729945e+06
7,284,Tete,Mensa-Annan,Sales Representative,2.312546e+06
8,285,Syed,Abbas,Pacific Sales Manager,1.725245e+05
9,290,Ranjit,Varkey Chudukatil,Sales Representative,4.509889e+06


## 8. Create a New Table from DataFrame
Creating new tables and adding them to our database is straightforward. \
We create/define our DataFrame and then we use the pandas.to_sql() method. \
We must define:
- the __name__ of the table
- the connection we are using (__con__)
- the __schema__ we want to create the table in (if not provided the table will be added the the default dbo schema) 
- if we want to have a separate index column or keep the columns as is (__index__)
- what SQLAlchemy should do if the table already exists (__if_exists__)

In [21]:
data = {'Sketch':['Cheese shop sketch', 'Silly walks', 'Spanish inquisition', 'Lumberjack song', 'Argument clinic'],
        'Length':['5:29', '4:05', '8:17', '2:41', '6:22']
       }

df2 = pd.DataFrame(data=data)
df2

,Sketch,Length
0,Cheese shop sketch,5:29
1,Silly walks,4:05
2,Spanish inquisition,8:17
3,Lumberjack song,2:41
4,Argument clinic,6:22


In [22]:
df2.to_sql(name='MontyPython', con=engine, schema='Sales', index=False, if_exists='replace')

5

In [23]:
# Save the table into the default dbo schema. dbo stands for DataBase Owner
df2.to_sql(name='MontyPython', con=engine, index=False, if_exists='replace')

5

## 9. Delete a table

In [ ]:
# Reading the table / Check existence
pd.read_sql("SELECT * FROM Sales.MontyPython", engine)

In [ ]:
# Executing the DROP TABLE command in Pandas
from pandas.io import sql
sql.execute('DROP TABLE IF EXISTS Sales.MontyPython', connection)

In [ ]:
# Reading the table / Check existence
pd.read_sql("SELECT * FROM Sales.MontyPython", engine)

## 10. Workflow example
In this part, we will go through an example of a workflow:
- Read: reading in data from the database into a Pandas DataFrame
- Update: modifying the data in the DataFrame
- Save: saving the modified DataFrame into a new database table

#### Reading in data

In [ ]:
# Read: reading in data from the database into a Pandas DataFrame
# Products ranked based on sold amount between 2011-07-01 and 2011-08-01
example_sql ="""
SELECT 
  DENSE_RANK() OVER (ORDER BY SUM(SOD.OrderQty) DESC) AS SalesRank,
  P.ProductID,
  P.Name,
  SUM(SOD.OrderQty) AS TotalSold
FROM 
  Production.Product AS P 
  JOIN Sales.SalesOrderDetail AS SOD ON P.ProductID = SOD.ProductID 
  JOIN Sales.SalesOrderHeader AS SOH ON SOD.SalesOrderID = SOH.SalesOrderID
WHERE 
  SOH.OrderDate BETWEEN '2011-07-01' AND '2011-07-31'
GROUP BY
  P.ProductID, P.Name
ORDER BY
  SUM(SOD.OrderQty) DESC
"""

example_df = pd.read_sql(example_sql, engine, index_col='SalesRank')
example_df

#### Modifying the DataFrame

In [ ]:
# Update: modifying the data in the DataFrame
# Adding a Goal column to th DataFrame, with the next month (August) sales goal of +10% TotalSold
example_df['August_Goal'] = round(example_df['TotalSold'] * 1.1, 0).astype(int)
example_df

#### Saving the modified DataFrame

In [ ]:
# Save: saving the modified DataFrame into a new database table
example_df.to_sql(name='August_Goal_Table', con=engine, schema='Sales', index=False, if_exists='replace')

#### Controlling the result

In [ ]:
# Double checking if the new table was created
check = pd.read_sql('SELECT * FROM Sales.August_Goal_Table', engine)
check